## Libraries

In [1]:
import import_ipynb
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
import Plots

importing Jupyter notebook from Plots.ipynb


## Gurobi Environment

In [2]:
# Gurobi environment. 
params = {
"WLSACCESSID": '656a0385-9063-4abe-9a9a-62c52d9bf86f',
"WLSSECRET": '965811b5-d564-49ae-bcfe-244106d2fad3',
"LICENSEID": 2404093,
}
env = gp.Env(params=params)

Set parameter Username
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2404093
Academic license - for non-commercial use only - registered to r.zhang-13@student.tudelft.nl


## Results Storing Function

In [3]:
def store_results(variable: gp.Var, ydim: int, xdim: int) -> np:
    '''
    Store the results from the matrix variable matrices in the numpy matrices.

    variable: The variable whose values will be stored.
    ydim, xdim: The dimensions of the variable.
    return: The numpy matrix storing the results.
    '''
    if ydim == 0:
      result = np.zeros((xdim))
      for col in range(xdim):
        result[col] = variable[col].X
        if result[col] < 1 and result[col] > -1:
            result[col] = 0
      return result

    result = np.zeros((ydim, xdim))
    for row in range(ydim):
      for col in range(xdim):
        result[row, col] = variable[row, col].X
        if result[row, col] < 1:
            result[row, col] = 0
    return result

## The Optimiser

In [4]:
def optimiser(parameters: dict, prediction_yield: list, prediction_demand: list, prediction_price: list, 
              opt_period: int, manual_demand: list=None, manual_price: list=None) -> dict:
    '''
    Strawberry supply chain optimiser.

    parameters: The parameters set for the optimiser.
    prediction_yield, prediction_demand, prediction_price: The predictions for the information for the next month.
    counter: Counting the number of optimisation times so far.
    opt_period: How many weeks will be considered in the optimiser.
    manual_demand, manual_price: Set the contract demand and price manually rather than as an optimised result.
    return: A dictionary containing all the results needed.
    '''
    LARGE = 1000000
    EPSILON = 0.01

    prediction_demand = np.array(prediction_demand)
    prediction_price = np.array(prediction_price)
    w = np.arange(0, opt_period) # Weeks
    t = np.arange(0, 7 * len(w)) # Days
    
    Th = parameters['Th'] # Harvest cycle.
    Tt = parameters['Tt'] # Transportation cycle.
    Zt = parameters['Zt'] # Transportation fee / kg.
    Zh = parameters['Zh'] # Holding cost fee /kg.
    Ztruck = parameters['Ztruck']
    theta_i = parameters['theta_i'] # (1 - theta_i) = deterioration speed in the producer's inventory.
    theta_t = parameters['theta_t'] # (1 - theta_t) = deterioration speed during the transportation.
    theta_r = parameters['theta_r'] # (1 - theta_r) = deterioration speed in the retailer's inventory.
    weightPrice_more = parameters['weightPrice_more'] # Sensitivity if contract demand > prediction demand.
    weightPrice_less = parameters['weightPrice_less']
    weightDiscount = parameters['weightDiscount'] # The discount portion for the price of strawberries sold in the discounted proce.
    weightShortage = parameters['weightShortage']
    capacity_inv = parameters['capacity_inv'] # The inventory capacity of the producer's inventory.
    capacity_truck = parameters['capacity_truck']
    F_proportion = parameters['F_proportion'] # Limitation of how many strawberries can be sold in the discounted price.

    batches = math.ceil(len(w)/Th)
    
    # Harvesting according to the yield.
    Y = prediction_yield
    H_week = np.zeros((batches, len(w)))
    for week in w:
      if week % Th == 0:
        H_week[week, week] = Y[week]
    H = np.zeros((batches, len(t)))
    for i in range(H_week.shape[0]):
      H[i, i * 7] = H_week[i, i]
  
    # Model optimisation
    model = gp.Model(env=env)
    model.Params.OutputFlag = 0
    model.params.NonConvex = 2
    model.Params.timeLimit = 1.0
    
    # Matrix variables.
    contract_demand = model.addVars(len(w), lb=0, vtype=GRB.CONTINUOUS)
    contract_price = model.addVars(len(w), lb=0, vtype=GRB.CONTINUOUS)

    weightPrice = model.addVars(len(w), lb=-1000, vtype=GRB.CONTINUOUS)
    demand_difference = model.addVars(len(w), lb=-1e5, vtype=GRB.CONTINUOUS)
    factor_quadratic = model.addVars(len(w), vtype=GRB.CONTINUOUS)
    
    left = model.addVars(batches, len(t), lb=0, vtype=GRB.CONTINUOUS)
    I = model.addVars(batches, len(t), lb=0, vtype=GRB.CONTINUOUS)
    T = model.addVars(batches, len(t), lb=0, vtype=GRB.CONTINUOUS)
    F = model.addVars(batches, len(t), lb=0, vtype=GRB.CONTINUOUS)
    left_tot = model.addVars(len(t), lb=0, vtype=GRB.CONTINUOUS)
    T_tot = model.addVars(len(t), lb=0, vtype=GRB.CONTINUOUS)
    F_tot = model.addVars(len(t), lb=0, vtype=GRB.CONTINUOUS)

    dispose = model.addVars(batches, len(t), lb=0, vtype=GRB.CONTINUOUS)
    dispose_tot = model.addVars(len(t), lb=0, vtype=GRB.CONTINUOUS)
    
    Truck = model.addVars(len(t), lb=0, vtype=GRB.INTEGER)

    retailer_demand = model.addVars(len(t), lb=0, vtype=GRB.CONTINUOUS)
    retailer_inv = model.addVars(len(t), lb=0, vtype=GRB.CONTINUOUS)

    ind_demandDifference = model.addVars(len(w), vtype=GRB.BINARY)
    ind_transport = model.addVars(batches, len(t), vtype=GRB.BINARY)
    ind_cap = model.addVars(len(t), vtype=GRB.BINARY)
    ind_dispose = model.addVars(batches, len(t), vtype=GRB.BINARY)
    ind_inventory = model.addVars(batches, len(t), vtype=GRB.BINARY)
    ind_left = model.addVars(batches, len(t), vtype=GRB.BINARY)
    
    Income_retailers = model.addVar(vtype=GRB.CONTINUOUS)
    Income_discount = model.addVar(vtype=GRB.CONTINUOUS)
    Holding_cost = model.addVar(vtype=GRB.CONTINUOUS)
    Transportation_cost = model.addVar(vtype=GRB.CONTINUOUS)
    Shortage_cost = model.addVar(vtype=GRB.CONTINUOUS)
    Dispose_cost = model.addVar(lb=0, vtype=GRB.CONTINUOUS)
    Deterioration_cost = model.addVar(lb=0, vtype=GRB.CONTINUOUS)

    # Add contract demand and contract price manually
    if manual_demand is not None:
       model.addConstrs(contract_demand[j] == manual_demand[j] for j in range(len(w)))
    
    # Decide contract price according to the contract demand.
    model.addConstrs(demand_difference[j] == contract_demand[j] - prediction_demand[j] for j in range(len(w)))
    model.addConstrs(demand_difference[j] >= LARGE * (ind_demandDifference[j] - 1) for j in range(len(w)))
    model.addConstrs(demand_difference[j] <= LARGE * ind_demandDifference[j] for j in range(len(w)))
    model.addConstrs(weightPrice[j] == weightPrice_more * ind_demandDifference[j] + weightPrice_less * (1 - ind_demandDifference[j]) for j in range(len(w)))
    model.addConstrs(factor_quadratic[j] == demand_difference[j] * demand_difference[j]/25 for j in range(len(w)))
    if manual_price is None:
      model.addConstrs((contract_price[j] == prediction_price[j] - factor_quadratic[j] * weightPrice[j])for j in range(len(w)))
    else:
      model.addConstrs(contract_price[j] == manual_price[j] for j in range(len(w)))
    
    # Inventory State
    model.addConstrs(left[i, 0] == H[i, 0] - T[i, 0] - F[i, 0] for i in range(batches))
    model.addConstrs(left[i, j] == I[i, j - 1] * theta_i + H[i, j] - T[i, j] - F[i, j] for i in range(batches) for j in range(1, len(t)))
    model.addConstrs(I[i, j] == left[i, j] - dispose[i, j] for i in range(batches) for j in range(len(t)))
    model.addConstrs(T_tot[j] == gp.quicksum(T[i, j] for i in range(batches)) for j in range(len(t)))
    model.addConstrs(F_tot[j] == gp.quicksum(F[i, j] for i in range(batches)) for j in range(len(t)))
    model.addConstrs(gp.quicksum(T_tot[i * 7 + j] for j in range(7)) <= contract_demand[i] for i in range(len(w)))
    model.addConstrs(gp.quicksum(F_tot[i * 7 + j] for j in range(7)) <= contract_demand[i] * F_proportion for i in range(len(w)))

    # Dispose Calculation
    model.addConstrs(left_tot[j] == gp.quicksum(left[i, j] for i in range(batches)) for j in range(len(t)))
    model.addConstrs(left_tot[j] >= capacity_inv + LARGE * (ind_cap[j] - 1) for j in range(len(t)))
    model.addConstrs(left_tot[j] <= capacity_inv + LARGE * ind_cap[j] for j in range(len(t)))
    model.addConstrs(dispose_tot[j] >= left_tot[j] - capacity_inv - LARGE * (1 - ind_cap[j]) for j in range(len(t)))
    model.addConstrs(dispose_tot[j] <= left_tot[j] for j in range(len(t)))
    model.addConstrs(gp.quicksum(dispose[i, j] for i in range(batches)) == dispose_tot[j] for j in range(len(t)))

    # T&F FIFO
    model.addConstrs(T[i, j] + F[i, j] >= - LARGE * (1 - ind_transport[i, j]) + EPSILON for i in range(batches) for j in range(len(t)))
    model.addConstrs(T[i, j] + F[i, j] <= LARGE * ind_transport[i, j] + EPSILON for i in range(batches) for j in range(len(t)))
    model.addConstrs(gp.quicksum(I[k, j] for k in range(i)) >= - LARGE * (1 - ind_inventory[i, j]) + EPSILON for i in range(1, batches) for j in range(len(t)))
    model.addConstrs(gp.quicksum(I[k, j] for k in range(i)) <= LARGE * ind_inventory[i, j] + EPSILON for i in range(1, batches) for j in range(len(t)))
    model.addConstrs(ind_transport[i, j] + ind_inventory[i, j] <= 1 for i in range(batches) for j in range(len(t)))

    # Dispose FIFO
    model.addConstrs(dispose[i, j] >= - LARGE * (1 - ind_dispose[i, j]) + EPSILON for i in range(batches) for j in range(len(t)))
    model.addConstrs(dispose[i, j] <= LARGE * ind_dispose [i, j] + EPSILON for i in range(batches) for j in range(len(t)))
    model.addConstrs(gp.quicksum(left[k, j] for k in range(i)) >= - LARGE * (1 - ind_left[i, j]) + EPSILON for i in range(batches) for j in range(len(w)))
    model.addConstrs(gp.quicksum(left[k, j] for k in range(i)) <= LARGE * ind_left[i, j] + EPSILON for i in range(batches) for j in range(len(w)))
    model.addConstrs(ind_dispose[i, j] + ind_left[i, j] <= 1 for i in range(batches) for j in range(len(t)))
    
    # Retailer Demand
    model.addConstrs(retailer_demand[j] == prediction_demand[j//7]/8 for j in range(len(t)))
    model.addConstrs(retailer_inv[j] == 0 for j in range(Tt))
    model.addConstrs(retailer_inv[j] == T_tot[j - Tt] * (theta_t ** Tt) + retailer_inv[j - 1] * theta_r for j in range(Tt, len(t)))
    model.addConstrs(retailer_inv[j] >= retailer_demand[j] for j in range(Tt, len(t)))

    model.addConstrs(Truck[j] * capacity_truck >= T_tot[j] + F_tot[j] for j in range(len(t)))
    
    model.addConstr(Income_retailers == gp.quicksum(contract_demand[j] * contract_price[j] for j in range(len(w))))
    model.addConstr(Income_discount == gp.quicksum(F_tot[j] * contract_price[int(j/7)] for j in range(len(t))) * weightDiscount)
    model.addConstr(Holding_cost == I.sum() * Zh)
    model.addConstr(Transportation_cost == (T_tot.sum() + F_tot.sum()) * Zt + Truck.sum() * Ztruck)
    model.addConstr(Shortage_cost == gp.quicksum((contract_demand[i] - gp.quicksum(T_tot[i * 7 + j] for j in range(7))) * prediction_price[i] * weightShortage for i in range(len(w))))
    model.addConstr(Dispose_cost == dispose_tot.sum())
    model.addConstr(Deterioration_cost == I.sum() * (1 - theta_i))

    model.setObjective(Income_retailers + Income_discount - Holding_cost - Transportation_cost - Shortage_cost * weightShortage - Dispose_cost - Deterioration_cost, GRB.MAXIMIZE)
    model.optimize()
    
    # Store the results.
    I = store_results(I, batches, len(t))
    T = store_results(T, batches, len(t))
    F = store_results(F, batches, len(t))
    T_tot = store_results(T_tot, 0, len(t))
    F_tot = store_results(F_tot, 0, len(t))
    contract_demand = store_results(contract_demand, 0, len(w))
    contract_price = store_results(contract_price, 0, len(w))
    Truck = store_results(Truck, 0, len(t))
    dispose = store_results(dispose, batches, len(t))
    retailer = store_results(retailer_inv, 0, len(t))

    Income_retailers = Income_retailers.X
    Income_discount = Income_discount.X
    Holding_cost = Holding_cost.X
    Transportation_cost = Transportation_cost.X
    Shortage_cost = Shortage_cost.X
    Dispose_cost = Dispose_cost.X
    Deterioration_cost = Deterioration_cost.X
    Contract_profit = model.ObjVal

    return {'Y':Y, 'H_week':H_week, 'H':H, 'I':I, 'T':T, 'T_tot':T_tot, 'F':F, 'F_tot':F_tot, 
            'contract_demand':contract_demand, 
            'contract_price':contract_price, 
            'prediction_price':prediction_price,
            'prediction_demand':prediction_demand,
            'Income_retailers':Income_retailers, 
            'Income_discount':Income_discount,
            'Transportation_cost':Transportation_cost, 
            'Shortage_cost':Shortage_cost, 
            'truck':Truck,
            'Profit':Contract_profit,
            'dispose':dispose,
            'retailer':retailer,
            'dispose_cost':Dispose_cost,
            'deterioration':Deterioration_cost}

## Execution Example

In [5]:
# demand_prediction = [2685.43017578125, 2673.31787109375, 2670.385498046875, 2668.9794921875]
# price_prediction = [7.97511163, 8.07187274, 8.07929887, 8.07096947]
# yield_prediction = [3616.96354167 * 1.2, 3486.45800781 * 1.2, 3418.65071615 * 1.2, 3330.74641927 * 1.2]

# parameters = {'Th': 1,
#               'Tt': 2,
#               'Zt': 0.1,
#               'Zh': 0.1,
#               'Ztruck': 100, 
#               'theta_i': 0.9,
#               'theta_t': 0.9,
#               'theta_r': 0.6,
#               'weightPrice_more':0.0001,
#               'weightPrice_less':-0.0001,
#               'weightDiscount':0.5,
#               'weightShortage': 2,
#               'capacity_inv': 5000,
#               'capacity_truck':500,
#               'F_proportion': 0.1
#               }
# def illustration(arrangement):

#     min_level = np.zeros((28)) # The minimum requirement level for the retailer.
#     for j in range(min_level.shape[0]):
#         min_level[j] = demand_prediction[j//7]/8 

#     print('The state of the producer\'s inventory after satisfying the demand is:\n')
#     Plots.plot_matrix(arrangement['I'], 100, size=(10, 5), title='Inventory State', xlabel='Days', ylabel='Batches', zlabel='Amount(Kg)', x=0.43)
#     Plots.plot_lines(arrangement['I'], title='Inventory State', xlabel='Days', ylabel='Amount(Kg)', label=['Batch0', 'Batch1', 'Batch2', 'Batch3'], show_label=1)
#     print('The transportation to the retailers is arranged as following images:\n')
#     Plots.plot_matrix(arrangement['T'], 500, size=(10, 5), title='Transportation Scheduling', xlabel='Days', ylabel='Batches', zlabel='Amount(Kg)', x=0.43)
#     Plots.plot_lines(arrangement['T'], title='Transporation Scheduling', xlabel='Days', ylabel='Amount(Kg)', label=['Batch0', 'Batch1', 'Batch2', 'Batch3'], show_label=1)
#     print('The transportation to the retailers in discounted price is arranged as following images:\n')
#     Plots.plot_matrix(arrangement['F'], 100, size=(10, 5), title='Discount Sales', xlabel='Days', ylabel='Batches', zlabel='Amount(Kg)', x=0.43)
#     Plots.plot_lines(arrangement['F'], title='Discount Sales', xlabel='Days', ylabel='Amount(Kg)', label=['Batch0', 'Batch1', 'Batch2', 'Batch3'], show_label=1)
#     Plots.plot_matrix(arrangement['dispose'], 100, size=(10, 5), title='Dispose Amount', xlabel='Days', ylabel='Batches', zlabel='Amount(Kg)', x=0.43)
#     Plots.plot_lines(np.concatenate((np.array(arrangement['retailer']), min_level), axis=0).reshape(2, -1), title='Retailer Inventory', xlabel='Days', ylabel='Amount(Kg)', label=['Strawberry Amount', 'Minimum Requirement Level'], show_label=1)
    
#     print('The yield is: ' + str(arrangement['Y']))
#     print('The weekly contract strawberry transportation amount for the next month is:' + str(arrangement['contract_demand']))
#     print('The weekly contract strawberry price for the next month is:' + str(arrangement['contract_price']))
#     print('The predicted demand for the next month is:' + str(arrangement['prediction_demand']))
#     print('The predicted price for the next month is:' + str(arrangement['prediction_price']))
#     print('The real transportation to the retailer in each day is:' + str(arrangement['T_tot'][0]))
#     print('The real transportation to the retailer in discounted price in each day is:' + str(arrangement['F_tot']))
#     print('The number of trucks arranged to deliver strawberries to the retailers in each day is:' + str(arrangement['truck']))
#     print('The expected total profit next month is:' + str(arrangement['Profit']))
#     print('The income by delivering the strawberry to the retailers is:' + str(arrangement['Income_retailers']))
#     print('The income by delivering the strawberry to the retailers in discounted price is:' + str(arrangement['Income_discount']))
#     print('The total transportation cost is:' + str(arrangement['Transportation_cost']))
#     print('The cost caused by failing to transport enough strawberry to the retailer is:' + str(arrangement['Shortage_cost']))
#     print('dispose:'+str(arrangement['dispose_cost']))
#     print('de:'+str(arrangement['deterioration']))

# strategy = optimiser(parameters, yield_prediction, demand_prediction, price_prediction, 4)
# illustration(strategy)
# baseline_strategy = optimiser(parameters, yield_prediction, demand_prediction, price_prediction, 4, manual_demand=demand_prediction, manual_price=price_prediction)
# illustration(baseline_strategy)